# KPIs:




## KPI 1
- The process of finding is suited KPI is shown below
- The final KPI is at the bottom of the section
- We considered different KPIs



- General Utilization (relative)
- ...

In [6]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go




# Lade das Datenset
charging_set = pd.read_csv("data/cleaned_charging_set_new.csv")
charging_set.head(3)



,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,userID,...,requestedDeparture_year,requestedDeparture_month,requestedDeparture_day,requestedDeparture_time,requestedDeparture_weekday,connectionDuration,chargingDuration,isRegistered,fullyCharged,parkingDuration
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,194.0,...,2020,1,2,20:51:54,3,6.039167,4.378056,1,1,1.66
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,4275.0,...,2020,1,2,23:31:50,3,9.025278,6.687500,1,1,2.34
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,344.0,...,2020,1,2,14:56:35,3,10.713056,2.641944,1,1,8.07


In [7]:
# Extract hour and calculate connection/charging time in hours
charging_set['connectionHour'] = charging_set['connectionTime'].dt.hour
total_stations = charging_set['stationID'].nunique()

hourly_utilization = charging_set.groupby('connectionHour').agg(
    TotalConnectionTime=('connectionDuration', 'sum'),
    TotalChargingTime=('chargingDuration', 'sum')
).reset_index()

# Convert durations to hours and calculate utilization ratios
hourly_utilization[['TotalConnectionTime_hours', 'TotalChargingTime_hours']] = hourly_utilization[['TotalConnectionTime', 'TotalChargingTime']] / 3600
hourly_utilization[['MeanConnectionUtilization', 'MeanChargingUtilization']] = hourly_utilization[['TotalConnectionTime_hours', 'TotalChargingTime_hours']].div(total_stations, axis=0)

# Plotting function
def create_utilization_plot(data, y_column, title, line_color):
    return px.line(data, x='connectionHour', y=y_column, title=title,
                   labels={'connectionHour': 'Hour of the day', y_column: 'Auslastung (0-1)'}
           ).update_traces(line_color=line_color).update_layout(
               title_font_size=18, legend_title_text="Legende", template="plotly_white",
               xaxis=dict(title="Hour of the day", tickmode='linear', tick0=0, dtick=1),
               yaxis=dict(title="Utilization (0-100%)", range=[0, 1])
           )

# Create and show plots
fig1 = create_utilization_plot(hourly_utilization, 'MeanConnectionUtilization', 'Average gross utilization per hour', '#00008B')
fig2 = create_utilization_plot(hourly_utilization, 'MeanChargingUtilization', 'Average net utilization per hour', '#FF8C00')

fig1.show()
fig2.show()


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
# Convert durations to hours and add time-based columns
charging_set['connectionDuration_hours'] = charging_set['connectionDuration'] / 3600
charging_set['chargingDuration_hours'] = charging_set['chargingDuration'] / 3600
charging_set['connectionDate'] = charging_set['connectionTime'].dt.date
charging_set['connectionWeek'] = charging_set['connectionTime'].dt.to_period('W').dt.start_time

def calculate_utilization(data, period_column):
    """Aggregate connection/charging times and compute utilization ratio."""
    utilization = data.groupby(period_column).agg(
        TotalConnectionTime=('connectionDuration_hours', 'sum'),
        TotalChargingTime=('chargingDuration_hours', 'sum')
    ).reset_index()
    utilization['UtilizationRatio'] = (utilization['TotalChargingTime'] / utilization['TotalConnectionTime']) * 100
    return utilization

def plot_utilization(data, period, chart_type, title, color):
    """Reusable function for plotting utilization ratio."""
    fig = go.Figure()
    trace = (
        go.Scatter(x=data[period], y=data['UtilizationRatio'], mode='lines', line=dict(color=color, width=2))
        if chart_type == 'line'
        else go.Bar(x=data[period], y=data['UtilizationRatio'], marker=dict(color=color, opacity=0.6))
    )
    fig.add_trace(trace)
    fig.update_layout(
        title=title,
        title_font_size=18,
        xaxis=dict(title=period.capitalize(), tickangle=45),
        yaxis=dict(title='Utilization Ratio (%)', range=[0, 100]),
        template="plotly_white"
    )
    fig.show()

# Calculate and plot utilization
plot_utilization(
    calculate_utilization(charging_set, 'connectionDate'),
    'connectionDate', 'line', 'Utilization Ratio (Net to Gross) per Day', '#1E90FF'
)
plot_utilization(
    calculate_utilization(charging_set, 'connectionWeek'),
    'connectionWeek', 'bar', 'Utilization Ratio (Net to Gross) per Week (Bar)', '#32CD32'
)


### KPI 2:
- shown below is the thought process of developing a second KPI
- The actual KPI used is the KPI at the bottom of the section

In [ ]:
# Ensure 'connectionTime' is a datetime object
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])

# Filter for registered users and add week/year for aggregation
registered_users = charging_set[charging_set['isRegistered'] == 1]
registered_users['week'] = registered_users['connectionTime'].dt.isocalendar().week
registered_users['year'] = registered_users['connectionTime'].dt.isocalendar().year

def aggregate_and_plot(data, title, color):
    """Aggregate unique users weekly and plot."""
    agg_data = (
        data.groupby(['year', 'week'])['userID']
        .nunique()
        .reset_index(name='UniqueUsers')
    )
    agg_data['YearWeek'] = pd.to_datetime(
        agg_data['year'].astype(str) + '-W' + agg_data['week'].astype(str) + '-1',
        format='%Y-W%U-%w'
    )
    fig = px.line(
        agg_data,
        x='YearWeek',
        y='UniqueUsers',
        title=title,
        labels={'YearWeek': 'Time', 'UniqueUsers': 'Unique Registered Users'}
    )
    fig.update_traces(line_color=color)
    fig.update_layout(
        title_font_size=18,
        xaxis=dict(
            title="Month",
            dtick="M1",
            tickformat="%b %Y",
            tickangle=-45,
        ),
        yaxis=dict(title="Unique Registered Users"),
        template="plotly_white"
    )
    return fig

# Generate plots
fig_all = aggregate_and_plot(
    registered_users,
    "Weekly Unique Registered Users Charging at EV Stations (All Sites)",
    color='#00008B'
)
fig_site1 = aggregate_and_plot(
    registered_users[registered_users['siteID'] == 1],
    "Weekly Unique Registered Users Charging at EV Stations (Site 1)",
    color='#FF8C00'
)
fig_site2 = aggregate_and_plot(
    registered_users[registered_users['siteID'] == 2],
    "Weekly Unique Registered Users Charging at EV Stations (Site 2)",
    color='#1f77b4'
)

# Show the graphs
fig_all.show()
fig_site1.show()
fig_site2.show()


In [ ]:
# Filter for registered users
registered_users = charging_set[charging_set['isRegistered'] == 1]

# Calculate the total connection and charging durations, and total number of sessions for registered users
total_connection_duration_registered = registered_users['connectionDuration'].sum()  # in hours
total_charging_duration_registered = registered_users['chargingDuration'].sum()  # in hours
total_sessions_registered = registered_users.shape[0]  # number of sessions (rows)

# Calculate the mean connection duration for registered users
mean_connection_duration_registered = total_connection_duration_registered / total_sessions_registered  # in hours

# Define the new KPI: average connections per user per week
# Step 1: Count connections per week per user
connections_per_week = registered_users.groupby(['userID', 'connectionWeek']).size()

# Step 2: Calculate the mean connections per week for each user
mean_connections_per_week_per_user = connections_per_week.groupby('userID').mean()

# Step 3: Calculate the average of these means across all users
mean_connections_per_week = mean_connections_per_week_per_user.mean()

# Calculate the total connection and charging durations, and total number of sessions for all users
total_connection_duration_all = charging_set['connectionDuration'].sum()  # in hours
total_charging_duration_all = charging_set['chargingDuration'].sum()  # in hours
total_sessions_all = charging_set.shape[0]  # number of sessions (rows)

# Calculate the mean connection and charging durations for all users (in hours)
mean_connection_duration_all = total_connection_duration_all / total_sessions_all  # in hours
mean_charging_duration_all = total_charging_duration_all / total_sessions_all  # in hours

# Define KPIs for tiles (new one for registered users, the last two for all)
kpis = {
    'Average connections per user per week (registered users)': mean_connections_per_week,
    'Average connection time (hours) (All users)': mean_connection_duration_all,
    'Average charging time (hours) (All users)': mean_charging_duration_all
}

# Visualization: Create tiles dynamically
fig = go.Figure()

# Create tiles for each KPI
for i, (title, value) in enumerate(kpis.items()):
    fig.add_trace(go.Indicator(
        mode="number",
        value=value,
        title={"text": title},
        domain={'x': [i/3, (i+1)/3], 'y': [0, 1]},
        number={'font': {'size': 40}, 'suffix': " Connections/Wk" if 'connections' in title.lower() else "h"}
    ))

# Update layout
fig.update_layout(
    template="plotly_white",
    title="KPI for Users",
    height=400,
    showlegend=False
)

# Show the visualization
fig.show()


# Observation
- The graph shows relatively high percentages of vehicles which are plugged in and are not charging
- The graph is also relatively volatile
- At 10:00 there is a siginifcant decrease, this probably aligns with people arriving at work with verhicles that are not fully charged

## KPI 3. Energy Consumption
- shown below is the process of developing a third KPI
- The final KPI is located at the bottom



- Energy Delivered by Session (Mean)
- Energy Requested by Session (Mean)
- Energy Consumed by Hour
- Max Energy Delivered within a session
- By Sites for all registered Users --> Times payment was required

In [ ]:
# Ensure connectionTime is datetime and extract the week
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])
charging_set['connectionWeek'] = charging_set['connectionTime'].dt.to_period('W').dt.to_timestamp()

# Weekly aggregation: energy delivered and session count
weekly_aggregation = charging_set.groupby('connectionWeek').agg(
    TotalEnergyDelivered=('kWhDelivered', 'sum'),
    TotalSessions=('sessionID', 'count')
).reset_index()

# Calculate per-session mean energy by week and overall
weekly_aggregation['MeanEnergyPerSession'] = weekly_aggregation['TotalEnergyDelivered'] / weekly_aggregation['TotalSessions']
overall_mean = charging_set['kWhDelivered'].sum() / charging_set['sessionID'].nunique()

# Calculate and round the mean energy requested per session
mean_energy_requested = round(charging_set['kWhRequested'].mean())

# Bar chart with swapped KPI annotations
fig = px.bar(
    weekly_aggregation,
    x='connectionWeek',
    y='MeanEnergyPerSession',
    title='Average energy delivered per session (kWh)',
    labels={'connectionWeek': 'Woche', 'MeanEnergyPerSession': 'Energy supplied (kWh)'},
    template="plotly_white",
    color_discrete_sequence=['#1f77b4']  # Set bar color
).update_layout(
    title_font_size=18,
    xaxis_title="Week",
    yaxis_title="Energy supplied (kWh)",
    yaxis_range=[0, weekly_aggregation['MeanEnergyPerSession'].max() * 1.1]
).add_annotation(
    text=f"Average energy requested per session: {mean_energy_requested} kWh",
    xref="paper", yref="paper",
    x=0, y=1.15,  # Now placed above the other annotation
    showarrow=False,
    font=dict(size=14, color="black", family="Arial"),
    align="left",
    xanchor="left"
).add_annotation(
    text=f"Average energy delivered per session: {overall_mean:.2f} kWh",
    xref="paper", yref="paper",
    x=0, y=1.05,  # Now placed below the first annotation
    showarrow=False,
    font=dict(size=14, color="black", family="Arial"),
    align="left",
    xanchor="left"
)

fig.show()


In [ ]:
# Ensure connectionTime is in datetime format and extract hour and date
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])
charging_set['connectionHour'] = charging_set['connectionTime'].dt.hour
charging_set['connectionDate'] = charging_set['connectionTime'].dt.date

# Group by date and hour, and calculate total energy delivered
energy_per_hour_per_day = charging_set.groupby(['connectionDate', 'connectionHour']).agg(
    TotalEnergyDelivered=('kWhDelivered', 'sum')
).reset_index()

# Second plot: with color differentiation by hour
fig2 = px.line(
    energy_per_hour_per_day,
    x='connectionDate',
    y='TotalEnergyDelivered',
    color='connectionHour',  # Different color for each hour
    title='Energy per hour for all days in the period (with color differentiation)',
    labels={'connectionDate': 'Datum', 'TotalEnergyDelivered': 'Energy supplied (kWh)', 'connectionHour': 'Hour of the day'},
    template="plotly_white"
)

fig2.update_layout(
    title_font_size=18,
    xaxis_title="Date",
    yaxis_title="Energy supplied (kWh)",
    xaxis=dict(tickangle=45),  # Rotate x-axis labels for better readability
    yaxis=dict(range=[0, energy_per_hour_per_day['TotalEnergyDelivered'].max() * 1.1])  # Slightly extend y-axis
)

# Show the plot
fig2.show()
